# Downloading Data / Setup

In [7]:
! pip install kaggle

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
! mkdir ~/.kaggle

In [10]:
! cp /content/drive/MyDrive/kaggle_api_credentials/kaggle.json ~/.kaggle/kaggle.json

In [11]:
! chmod 600 ~/.kaggle/kaggle.json

In [12]:
! kaggle competitions download challenges-in-representation-learning-facial-expression-recognition-challenge --force

 77% 221M/285M [00:00<00:00, 772MB/s] 
100% 285M/285M [00:00<00:00, 375MB/s]


In [13]:
!ls -lh /content

total 286M
-rw-r--r-- 1 root root 286M Dec 11  2019 challenges-in-representation-learning-facial-expression-recognition-challenge.zip
drwx------ 6 root root 4.0K Jun  5 13:58 drive
drwxr-xr-x 1 root root 4.0K Jun  3 14:04 sample_data
drwxr-xr-x 4 root root 4.0K Jun  5 13:57 wandb


In [14]:
! unzip challenges-in-representation-learning-facial-expression-recognition-challenge

Archive:  challenges-in-representation-learning-facial-expression-recognition-challenge.zip
  inflating: example_submission.csv  
  inflating: fer2013.tar.gz          
  inflating: icml_face_data.csv      
  inflating: test.csv                
  inflating: train.csv               


# Set up Wandb

In [15]:
!pip install -q wandb

In [16]:
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [17]:
import random

import wandb

# Start a new wandb run to track this script.
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="ashar-22-free-university-of-tbilisi-",
    # Set the wandb project where this run will be logged.
    project="setup",
    # Track hyperparameters and run metadata.
    config={
        "learning_rate": 0.02,
        "architecture": "CNN",
        "dataset": "CIFAR-100",
        "epochs": 10,
    },
)

# Simulate training.
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2**-epoch - random.random() / epoch - offset
    loss = 2**-epoch + random.random() / epoch + offset

    # Log metrics to wandb.
    run.log({"acc": acc, "loss": loss})

# Finish the run and upload any remaining data.
run.finish()

acc,▁▁▁▆▇▄▆█
loss,█▄▂▂▂▁▁▂
acc,0.84463
loss,0.2518


# Handling Data

In [ ]:
# utils/data.py
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import numpy as np
from torchvision import transforms
import wandb
from PIL import Image

class FER2013Dataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        pixels = np.array(self.data.iloc[idx]['pixels'].split(), dtype='uint8').reshape(48, 48)
        image = Image.fromarray(pixels)  # PIL Image
        label = int(self.data.iloc[idx]['emotion']) if 'emotion' in self.data.columns else -1

        if self.transform:
            image = self.transform(image)
        return image, label


def get_data_loaders(csv_path, batch_size=64):
    # Define transforms
    train_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.RandomResizedCrop(48, scale=(0.8, 1.0)),
        transforms.ColorJitter(brightness=0.3, contrast=0.3),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    val_test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    # Load the full dataset without a transform initially
    dataset = FER2013Dataset(csv_file=csv_path, transform=None)

    # Split into train/val/test
    train_len = int(0.7 * len(dataset))
    val_len = int(0.15 * len(dataset))
    test_len = len(dataset) - train_len - val_len
    train_data, val_data, test_data = random_split(dataset, [train_len, val_len, test_len])

    # Manually assign transforms to each subset
    train_data.dataset.transform = train_transform
    val_data.dataset.transform = val_test_transform
    test_data.dataset.transform = val_test_transform

    # Create data loaders
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size)
    test_loader = DataLoader(test_data, batch_size=batch_size)

    return train_loader, val_loader, test_loader


# Adding Layers

In [71]:
import torch.nn as nn
import torch.nn.functional as F
import torch.nn as nn
import torch.nn.functional as F

class DeepCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 64, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)

        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)

        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)

        self.conv4 = nn.Conv2d(256, 256, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)

        self.conv5 = nn.Conv2d(256, 512, 3, padding=1)
        self.bn5 = nn.BatchNorm2d(512)

        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(512 * 1 * 1, 256)
        self.fc2 = nn.Linear(256, 7)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))  # 48 -> 24
        x = self.pool(F.relu(self.bn2(self.conv2(x))))  # 24 -> 12
        x = self.pool(F.relu(self.bn3(self.conv3(x))))  # 12 -> 6
        x = self.pool(F.relu(self.bn4(self.conv4(x))))  # 6 -> 3
        x = self.pool(F.relu(self.bn5(self.conv5(x))))  # 3 -> 1
        x = x.view(-1, 512 * 1 * 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


# Training

In [72]:
# train.py
import torch
import torch.nn as nn
import torch.optim as optim
import wandb

def train_and_validate(csv_path, batch_size=64, lr=0.001, epochs=10):
    wandb.init(project="facial-expression", name="run_4_deep_cnn", config={
        "model_name" : "DeepCNN",
        "batch_size": batch_size,
        "lr": lr,
        "epochs": epochs
    }, reinit=True)

    wandb.config.update({
    "optimizer": "SGD with momentum",
    "scheduler": "CosineAnnealingLR",
    "regularization": "Dropout + L2",
    "data_augmentation": "flip, rotate, crop, color jitter"
    })

    try:
        config = wandb.config

        train_loader, val_loader, test_loader = get_data_loaders(csv_path, config.batch_size)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = DeepCNN().to(device)
        wandb.watch(model, log="all", log_freq=10)  # logs gradients and parameters

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=config.lr, momentum=0.9, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

        for epoch in range(config.epochs):
            model.train()
            train_loss = 0
            correct = 0
            total = 0

            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            val_loss = 0
            val_correct = 0
            val_total = 0
            model.eval()

            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item()
                    _, predicted = torch.max(outputs.data, 1)
                    val_total += labels.size(0)
                    val_correct += (predicted == labels).sum().item()

            train_loss_avg = train_loss / len(train_loader)
            train_acc = correct / total
            val_loss_avg = val_loss / len(val_loader)
            val_acc = val_correct / val_total

            # print(f"Logging: train_acc={train_acc}, val_acc={val_acc}, lr={scheduler.get_last_lr()[0]}")

            wandb.log({
                "epoch": epoch + 1,
                "train_loss": train_loss_avg,
                "train_accuracy": train_acc,
                "val_loss": val_loss_avg,
                "val_accuracy": val_acc,
                "learning_rate": scheduler.get_last_lr()[0]
            })

            print(f"Epoch {epoch + 1}/{config.epochs} | "
                  f"Train Loss: {train_loss_avg:.4f}, Train Acc: {train_acc:.4f} | "
                  f"Val Loss: {val_loss_avg:.4f}, Val Acc: {val_acc:.4f}")
            scheduler.step()

        # Evaluate on test set
        test_loss = 0
        test_correct = 0
        test_total = 0
        model.eval()
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                test_total += labels.size(0)
                test_correct += (predicted == labels).sum().item()

        test_loss_avg = test_loss / len(test_loader)
        test_acc = test_correct / test_total

        print(f"\n✅ Final Test Loss: {test_loss_avg:.4f}, Test Accuracy: {test_acc:.4f}")

        wandb.log({
            "test_loss": test_loss_avg,
            "test_accuracy": test_acc
        })

        # Save model
        torch.save(model.state_dict(), "model.pth")
        wandb.save("model.pth")  # optional but recommended
        artifact = wandb.Artifact('facial-expression-model', type='model')
        artifact.add_file('model.pth')
        wandb.log_artifact(artifact)
    finally:
        wandb.finish()

if __name__ == "__main__":
    train_and_validate("train.csv")

Logging: train_acc=0.3338972929936306, val_acc=0.39526242452392013, lr=0.001
Epoch 1/10 | Train Loss: 1.6609, Train Acc: 0.3339 | Val Loss: 1.5581, Val Acc: 0.3953
Logging: train_acc=0.4557125796178344, val_acc=0.4633070134695773, lr=0.0009755282581475768
Epoch 2/10 | Train Loss: 1.4136, Train Acc: 0.4557 | Val Loss: 1.4141, Val Acc: 0.4633
Logging: train_acc=0.518312101910828, val_acc=0.5134695773339526, lr=0.0009045084971874736
Epoch 3/10 | Train Loss: 1.2622, Train Acc: 0.5183 | Val Loss: 1.2871, Val Acc: 0.5135
Logging: train_acc=0.5774283439490446, val_acc=0.5281003251277288, lr=0.0007938926261462366
Epoch 4/10 | Train Loss: 1.1307, Train Acc: 0.5774 | Val Loss: 1.2473, Val Acc: 0.5281
Logging: train_acc=0.637937898089172, val_acc=0.5141662796098467, lr=0.0006545084971874737
Epoch 5/10 | Train Loss: 0.9899, Train Acc: 0.6379 | Val Loss: 1.2710, Val Acc: 0.5142
Logging: train_acc=0.6924761146496815, val_acc=0.5366929865304226, lr=0.0005
Epoch 6/10 | Train Loss: 0.8397, Train Acc: 0

epoch,▁▂▃▃▄▅▆▆▇█
learning_rate,██▇▇▆▄▃▂▂▁
test_accuracy,▁
test_loss,▁
train_accuracy,▁▃▃▄▅▆▆▇██
train_loss,█▇▆▅▄▃▃▂▁▁
val_accuracy,▁▄▆▇▆▇▆███
val_loss,█▅▂▁▂▂▄▁▁▁
epoch,10
learning_rate,2e-05
test_accuracy,0.58533
